In [16]:
from pathlib import Path
import solopy
import logging

In [13]:
# log file directory
LOGDIR = Path("./log") 
LOGDIR.mkdir(parents=True, exist_ok=True)
fpath_log_lv0 = LOGDIR/'lv0_example.log' # log file path for Lv0
if fpath_log_lv0.exists():
    fpath_log_lv0.unlink() # remove existing log file

fpath_log_lv1 = LOGDIR/'lv1_example.log' # log file path for Lv1
if fpath_log_lv1.exists():
    fpath_log_lv1.unlink() # remove existing log file

fpath_log_comb = LOGDIR/'combmaster_example.log' # log file path for CombMaster    

# Data directory
DATADIR = Path("../solo-data")
MASTERDIR = DATADIR / "calibration_files"
MASTERDIR.mkdir(parents=True, exist_ok=True)

subdir = "2025_0717"

# Lv0 Data directory
Lv0DIR = DATADIR / "Lv0"
Lv0_subdir = Lv0DIR / subdir

# Lv1 Data directory
Lv1DIR = DATADIR / "Lv1"
Lv1_subdir = Lv1DIR / subdir  
Lv1_subdir.mkdir(parents=True, exist_ok=True)

Lv1_wcsdir = DATADIR / "Lv1_wcs"
Lv1_wcsdir.mkdir(parents=True, exist_ok=True)

In [14]:
# Initialize Lv0 class
lv0 = solopy.Lv0(log_file=fpath_log_lv0)

# Initialize Lv1 class
lv1 = solopy.Lv1(log_file=fpath_log_lv1)

# Initialize CombMaster class
comb = solopy.CombMaster(log_file=fpath_log_comb)

In [18]:
# --- 0. 기본 설정 ---
# solopy 로거 설정 (이미 클래스 내부에 설정되어 있지만,
# 메인 스크립트의 로그도 함께 보려면 설정)
logging.basicConfig(level=logging.INFO,
                    format="%(asctime)s [%(levelname)s] %(message)s",
                    handlers=[
                        logging.StreamHandler(), # 콘솔 출력
                        logging.FileHandler(LOGDIR/"solopy-example.log") # 파일 출력
                    ])
logger = logging.getLogger(__name__)

In [ ]:
# Update header (LV0)
for f in Lv0_subdir.glob("*.fits*"):
    lv0.update_header(f)

2025-10-27 15:52:38,808 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_002.fits.bz2
2025-10-27 15:52:38,808 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_002.fits.bz2
2025-10-27 15:52:41,210 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_003.fits.bz2
2025-10-27 15:52:41,210 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dk03_003.fits.bz2
2025-10-27 15:52:43,724 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/CARLOVA_006_20250717091050.fits.bz2
2025-10-27 15:52:43,724 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/CARLOVA_006_20250717091050.fits.bz2
2025-10-27 15:52:46,121 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dark007_001.fits.bz2
2025-10-27 15:52:46,121 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/dark007_001.fits.bz2
2025-10-27 15:52:48,667 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_0717/ELENORA_010_20250717054251.fits.bz2
2025-10-27 15:52:48,667 [INFO] Updated LV0 header: ../solo-data/Lv0/2025_071

In [ ]:
try:
    # 2-1. 마스터 바이어스 생성
    # Lv0 디렉터리에서 IMAGETYP='BIAS'인 파일 찾기 (파일명으로 검색)

    bias_files = list(Lv0_subdir.glob("*bias*.fits.bz2"))
    if bias_files:
        comb.make_mbias(bias_files, MASTERDIR)
    else:
        logger.warning("No bias files found to combine.")

    # 2-2. 마스터 다크 생성
    # Lv0 디렉터리에서 IMAGETYP='DARK'인 파일 찾기 (파일명으로 검색)
    logger.info("--- Starting Master Dark ---")
    dark_files = list(Lv0_subdir.glob("*dark*.fits.bz2")) + \
                 list(Lv0_subdir.glob("*dk*.fits.bz2"))
    if dark_files:
        comb.make_mdark(dark_files, MASTERDIR, key_exptime='EXPTIME')
    else:
        logger.warning("No dark files found to combine.")

    # 2-3. 마스터 플랫 생성 (R 필터)
    # (참고: 실제 Flat 파일이 없다면 SxLC_01_MasterFrame.py처럼 과학 프레임을 대신 사용)
    logger.info(f"--- Starting Master Flat ({TARGET_FILTER}) ---")
    # 예: 파일명에 'flat'과 필터 이름(r)이 들어간 경우
    flat_files = list(LV0_DATA_DIR.glob(f"*flat*{TARGET_FILTER}*.fits.bz2"))
    
    # 만약 flat 파일이 없다면, 과학 프레임(Light)을 대신 사용할 수 있습니다.
    # if not flat_files:
    #     logger.warning("No flat files found. Using LIGHT frames as pseudo-flats.")
    #     light_files = [f for f in LV0_DATA_DIR.glob("*.fits.bz2") 
    #                    if 'bias' not in f.name and 'dark' not in f.name and 'dk' not in f.name]
    #     flat_files = light_files # (실제로는 정렬, 필터링 등이 더 필요할 수 있음)

    if flat_files:
        comb.make_mflat(flat_files, MASTERDIR, filter_name=TARGET_FILTER, key_exptime='EXPTIME')
    else:
        logger.warning(f"No {TARGET_FILTER}-filter flat files found.")

except Exception as e:
    logger.error(f"Master frame creation failed: {e}")

In [ ]:
import solopy
from pathlib import Path




# --- 1. 경로 설정 ---
# (경로는 예시이므로 실제 환경에 맞게 수정하세요)
BASE_DIR = Path(".") # 현재 스크립트가 실행되는 위치
DATADIR = Path("../solo-data") # 데이터 상위 폴더

# Lv0 헤더가 업데이트된 파일이 있는 폴더
LV0_DATA_DIR = DATADIR / "Lv0" / "2025_0717"


# 사용할 필터 (예: 'R' 필터)
TARGET_FILTER = 'R' # 이 필터 이름은 파일명 검색 및 mflat 생성에 사용됩니다.

logger.info(f"Using Lv0 data from: {LV0_DATA_DIR}")
logger.info(f"Master frames will be saved to: {MASTERDIR}")
logger.info(f"Lv1 WCS output to: {Lv1_wcsdir}")
logger.info(f"Lv1 Processed output to: {LV1_PROC_DIR}")

# --- 2. CombMaster: 마스터 프레임 생성 ---
# CombMaster 초기화 (solopy 내부 로거 사용)
comb = solopy.CombMaster(log_file="combmaster.log")




# --- 3. Lv1: 과학 프레임 처리 ---
# Lv1 초기화 (solopy 내부 로거 사용)
lv1 = solopy.Lv1(log_file="lv1_processing.log")

# 3-1. 처리할 과학(Light) 프레임 목록 가져오기
# (파일명에 bias, dark, dk, flat이 포함되지 않은 모든 파일)
logger.info("--- Starting Lv1 Processing ---")
science_files = [f for f in LV0_DATA_DIR.glob("*.fits.bz2")
                 if 'bias' not in f.name.lower() and \
                    'dark' not in f.name.lower() and \
                    'dk' not in f.name.lower() and \
                    'flat' not in f.name.lower()]

if not science_files:
    logger.warning("No science files found in Lv0 directory.")
else:
    logger.info(f"Found {len(science_files)} science frames for Lv1 processing.")

    # 3-2. WCS 업데이트
    # (Lv0 파일 -> Lv1_WCS_DIR)
    wcs_updated_files = []
    for fpath in science_files:
        try:
            logger.info(f"Processing WCS for: {fpath.name}")
            wcs_path = lv1.update_wcs(fpath, Lv1_wcsdir)
            if wcs_path:
                wcs_updated_files.append(wcs_path)
        except Exception as e:
            logger.error(f"Failed WCS update for {fpath.name}: {e}")

    # 3-3. 전처리 (Bias, Dark, Flat 보정)
    # (Lv1_WCS_DIR 파일 -> Lv1_PROC_DIR)
    if not wcs_updated_files:
         logger.warning("No WCS files to process (WCS step failed or produced no files).")
    else:
        logger.info(f"Starting preprocessing for {len(wcs_updated_files)} WCS-updated files...")
        for fpath_wcs in wcs_updated_files:
            try:
                logger.info(f"Preprocessing: {fpath_wcs.name}")
                lv1.preprocessing(fpath_wcs, LV1_PROC_DIR, MASTERDIR)
            except Exception as e:
                logger.error(f"Failed preprocessing for {fpath_wcs.name}: {e}")

logger.info("--- Pipeline Finished ---")